In [ ]:
import tspart
from IPython.display import display

Import the image and get it's size.

In [ ]:
img = tspart.load_image_as_array("tests\\bowser.jpg")
size = tspart.rgb_image_array_size(img)

Split into CMYK channels.

In [ ]:
cmyk = tspart.split_cmyk(img)

Stipple the images.

In [ ]:
cmyk_points = tspart.voronoi.stipple_image_multi(cmyk, 100)

Solve the TSP problens with a free online solver.

In [ ]:
neos = tspart.neos.get_client()
jobs = tspart.neos.submit_solves(neos, "yourname@example.com", cmyk_points)
solves = tspart.neos.get_solves_blocking(neos, jobs, 0.1)

Map the indecies back to actual points.

In [ ]:
cmyk_routes = tspart.map_points_to_route_multi(cmyk_points, solves)

Save the routes locally for later use.

In [ ]:
tspart.save_routes("tests\\bowser.json", cmyk_routes, size)

Draw a bitmap image.

In [ ]:
drawn_image = tspart.draw_cmyk_routes(cmyk_points, size, 2)

Display the image.

In [ ]:
width = 600

ratio = size[0] / size[1]
new_size = (
    width,
    int(round(width / ratio))
)

display(drawn_image.resize(new_size))